In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import glob
import time
from sklearn.model_selection import train_test_split
import xgboost


In [2]:
%autosave 60

Autosaving every 60 seconds


In [56]:
list_features=["IMEI","sp", "EVVSP", "EVGPO", "EVACP", "EVODO",
             "EVOAS", "EVACO_Z", "EVSOH", "EVBMA_Latest", "EVBMA_Max",
             "EVBMA_Min", "EVIGM_Latest", "EVCOM_Latest", "EVCFN", "EVTRQ", 
             "EVBBV", "EVDR1", "EVDR2", "EVACP", "EVHVS", "EVPWA_MAX", 
             "EVPWA_MIN", "EVBMI_Latest", "EVSOMA", "EVSOMI", "EVIGM_Max",
             "EVIRT_Max", "EVIRT_Min", "EVIDC", "EVMGT", "EVMGS", "EVMGF", "EVMGR",
             "EVIND", "EVICM", "EVDI1", "EVDI2", "EVDIT", "EVDOA", "EVACS", "EVMSC",
             "EVRER", "EVDRV", "EVMTR", "EVTRE", "EVBFN", "EVHTP_AVG",
             "EVSMA_MAX"]

list_features=list(set(list_features))

In [57]:
%%time
path = r'/home/cwshpmu2282/Desktop/EVRangePrediction/data/raw/5_vehicle_data_model_testing'
all_files = glob.glob(path + "/*.csv")

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df = df[list_features]
    li.append(df)
    
data = pd.concat(li, axis=0, ignore_index=True)


CPU times: user 51.8 s, sys: 4.55 s, total: 56.4 s
Wall time: 31.2 s


In [58]:
data = data.dropna(axis=1)

In [59]:
%%time

X = data.drop(columns = ["EVSMA_MAX"], axis = 1)
y = data.EVSMA_MAX.values

CPU times: user 125 ms, sys: 234 ms, total: 359 ms
Wall time: 356 ms


In [60]:
x_train,x_test,y_train,y_test=train_test_split(X, y, test_size=0.33, random_state=42)

In [61]:
xgb_train = xgboost.DMatrix(x_train, label=y_train)
xgb_test = xgboost.DMatrix(x_test, label = y_test)

In [68]:
params= {
    "eta": 0.002,
    "max_depth": 3,
    "subsample": 0.5
    
    
}

In [69]:
model_train=xgboost.train(params, xgb_train, 10000, evals = [(xgb_test, "test" )], verbose_eval= 1000)

[0]	test-rmse:73.8026
[1000]	test-rmse:10.4831
[2000]	test-rmse:2.53135
[3000]	test-rmse:1.71628
[4000]	test-rmse:1.50925
[5000]	test-rmse:1.39683
[6000]	test-rmse:1.3291
[7000]	test-rmse:1.28031
[8000]	test-rmse:1.24331
[9000]	test-rmse:1.21425
[9999]	test-rmse:1.19057


In [1]:
xgboost.plot_importance(model_train, importance_type = "cover")

plt.rcParams["figure.figsize"] = (16, 16)
plt.show()
plt.savefig('xgb_plot_imp.png')

NameError: name 'xgboost' is not defined

In [ ]:
import pickel
pickel.dump(model_train,open('/home/cwshpmu2282/Desktop/EVRangePrediction/model/xgb.pickel.dat'),'rb')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rnd_clf = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42)
rnd_clf.fit(X,y)

importances = rnd_clf.feature_importances_

for name, importance in zip(X, rnd_clf.feature_importances_):
    print(name, "=", importance)